Spotify-Lab-Basic-OOP section

In [5]:
class Song:
  total_songs = 0  #This is a static variable, stays attached to the class

  def __init__(self, song_id, title, artist, duration):
    self.__song_id = song_id
    self.__title = title
    self.__artist = artist
    self.__duration = duration
    self.__play_count = 0

    Song.total_songs += 1

  def get_title(self):
    return self.__title

  def get_artist(self):
    return self.__artist

  def set_duration(self, new_duration):
    if new_duration > 0:
      self.__duration = new_duration
      return True
    print("Duration must be positive!")
    return False

  def __increment_plays(self):
    self.__play_count += 1

  def play(self):
    self.__increment_plays()
    print(f"Playing {self.__title} (Plays: {self.__play_count})")

  @staticmethod
  def get_total_songs():
    return Song.total_songs

class Playlist:
  def __init__(self, name, creator):
    self.__name = name
    self.__creator = creator
    self.songs = []

  def add_song(self, song):
    self.songs.append(song)
    print(f"{song.get_title()} added to {self.__name}")

  def display(self):
    print(f"\nPlaylist: {self.__name}")
    for i, song in enumerate(self.songs, 1): # while typing this, I had no idea how this works. I recognize it is a for-loop but it looks different.
      print(f"{i}. {song.get_title()} - {song.get_artist()}")

class User:
  total_users = 0

  def __init__(self, username, email, balance):
    self.__username = username
    self.__email = email
    self.__balance = balance
    self.__playlists = []

    User.total_users += 1

  def pay_for_premium(self, cost):
    if cost <= self.__balance:
      self.__balance -= cost
      print("Premium activated!")
      return True
    print("Insufficient balance!")
    return False

  def create_playlist(self, name):
    playlist = Playlist(name, self.__username)
    self.__playlists.append(playlist)
    return playlist

  @staticmethod
  def get_total_users():
    return User.total_users

In [6]:
song1 = Song(101, "Blinding Lights", "The Weeknd", 200)
song2 = Song(102, "Shape of You", "Ed Sheeran", 234)

print("Total Songs:", Song.get_total_songs())

# play song
song1.play()
song2.play()

# create user
user1 = User("Amar", "amar@spotify.com", 500)

# create playlist
favorites = user1.create_playlist("My Favorites")

# add song objects to playlist
favorites.add_song(song1)
favorites.add_song(song2)

# display playlists
favorites.display()

# pay for premium
user1.pay_for_premium(200)


Total Songs: 2
Playing Blinding Lights (Plays: 1)
Playing Shape of You (Plays: 1)
Blinding Lights added to My Favorites
Shape of You added to My Favorites

Playlist: My Favorites
1. Blinding Lights - The Weeknd
2. Shape of You - Ed Sheeran
Premium activated!


True

Spotify-Lab-Advanced-OOP Section

In [7]:
def playback_logger(func):
  """DECORATOR: Adds logging around playback"""
  def wrapper(self):
    print("    Logging playback...")
    result = func(self)
    print("    Playback complete\n")
  return wrapper

class Song:
  total_songs = 0

  def __init__(self, song_id, title, artist, duration):
    self.__song_id = song_id
    self.__title = title
    self.__artist = artist
    self.__duration = duration
    self.__play_count = 0
    Song.total_songs += 1

  def get_title(self):
    return self.__title

  def get_artist(self):
    return self.__artist

  def __increment_plays(self):
    self.__play_count += 1

  @playback_logger
  def play(self):
    self.__increment_plays()
    print(f"Playing: {self.__title} ({self.__play_count})")

class Playlist:
  def __init__(self, name):
    self.__name = name
    self.songs = []

  def add_song(self, song):
    self.songs.append(song)

  def __iter__(self):
    """ITERATOR ENTRY POINT"""
    self._index = 0
    return self

  def __next__(self):
    """ITERATOR CONTROL"""
    if self._index >= len(self.songs):
      raise StopIteration # did not know what "raise" does
    song = self.songs[self._index]
    self._index += 1
    return song

  def play_songs(self):
    """GENERATOR: Yields songs one at a time"""
    for song in self.songs:
      yield song

  def play_all(self, player):
    """
    DUCK TYPING:
    Any object with a play() method works
    """
    for song in self:
      player.play(song)

def playback_limiter(max_plays):
  """
  CLOSURE:
  Remembers play count across calls
  """
  count = 0

  def limiter(song):
    nonlocal count
    if count < max_plays:
      count += 1
      song.play()
    else:
      print("    Playback limit reached!")
  return limiter

class SimplePlayer:
  def play(self, song):
    song.play()

class LimitedPlayer:
  def __init__(self, limiter):
    self.limiter = limiter

  def play(self, song):
    self.limiter(song)


In [8]:
s1 = Song(101, "Blinding Lights", "The Weeknd", 200)
s2 = Song(102, "Shape of You", "Ed Sheeran", 234)

# playlist
favorites = Playlist("My Favorites")
favorites.add_song(s1)
favorites.add_song(s2)

# closure
limit_1 = playback_limiter(1)

# players
normal_player = SimplePlayer()
limited_player = LimitedPlayer(limit_1)

# duck typing in action
favorites.play_all(normal_player)
print(" ")
favorites.play_all(limited_player)

    Logging playback...
Playing: Blinding Lights (1)
    Playback complete

    Logging playback...
Playing: Shape of You (1)
    Playback complete

 
    Logging playback...
Playing: Blinding Lights (2)
    Playback complete

    Playback limit reached!
